In [3]:
import requests
import pandas as pd

# Overpass API URL
OVERPASS_URL = "http://overpass-api.de/api/interpreter"

# Function to fetch data from Overpass API
def fetch_overpass_data(query):
    try:
        response = requests.get(OVERPASS_URL, params={'data': query})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Function to process OSM elements into a DataFrame
def process_elements(elements):
    data = []
    for elem in elements:
        # Extracting common details (id, lat, lon, name)
        details = {
            'id': elem.get('id'),
            'lat': elem.get('lat'),
            'lon': elem.get('lon'),
            'name': elem.get('tags', {}).get('name', 'Unknown')
        }
        # Add more tags here if needed
        data.append(details)
    return pd.DataFrame(data)

# Queries for different POI types
queries = {
    'parks': 'nwr["leisure"="park"](bbox);',
    'playgrounds': 'nwr["leisure"="playground"](bbox);',
    'supermarkets': 'nwr["shop"="supermarket"](bbox);'
}

# Replace bbox with the bounding box of your area of interest, format: south,west,north,east
bbox = "53.389881, -113.724292, 53.646266, -113.081592"  # Example bbox, replace with your area

# Fetch and process data
all_data = {}
for poi_type, query in queries.items():
    full_query = f'[out:json];{query.replace("bbox", bbox)}out;'
    data = fetch_overpass_data(full_query)
    if data:
        elements = data.get('elements', [])
        df = process_elements(elements)
        all_data[poi_type] = df

        # Save each POI type to a CSV file
        df.to_csv(f'{poi_type}.csv', index=False)
    else:
        print(f"Failed to fetch data for {poi_type}")

In [4]:
import leafmap

ModuleNotFoundError: No module named 'leafmap'